In [36]:
from src.DataManipulation.DataLoader import get_dataloaders
from utils import save_img

from src.model.model import MyBigModel as NewBigModel
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
# from skimage.metrics import structural_similarity as ssim
import torch
from pytorch_msssim import ssim

import os
model_dir = "fflMix-0.0002-NewBigModel-1746968864.9918458-Wavelet/"
for model_name in os.listdir(model_dir):
    model_path = f"fflMix-0.0002-NewBigModel-1746968864.9918458-Wavelet/{model_name}"
    raw_dir = "uw_data/uw_data/train/a"
    ref_dir = "uw_data/uw_data/train/b"
    test_raw_dir = "uw_data/uw_data/test/a"
    test_ref_dir = "uw_data/uw_data/test/b"
    batch_size=1
    num_workers=4


    device = "cuda"
    #model = SpectralTransformer().to(device)
    #model = NewModel().to(device)
    model = NewBigModel().to(device)
    #model = torch.load(model_path, map_location=device, weights_only=False)
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    psnr_values = []
    ssim_values = []
    _,test_loader = get_dataloaders(raw_dir,ref_dir, test_raw_dir,test_ref_dir, batch_size)
    results_dir = 'evaluation_results'
    os.makedirs(results_dir, exist_ok=True)

    with torch.no_grad():
        for idx, (raw_img,ref_img) in enumerate(test_loader):
            raw_img = raw_img.to(device)
            ref_img = ref_img.to(device)

            enhanced_img = model(raw_img)

            enhanced_np = enhanced_img[0].squeeze().detach().cpu()
            save_img(enhanced_np, f"test/{idx}.png")
            enhanced_np = enhanced_np.numpy()
            enhanced_np = enhanced_np.transpose((1, 2, 0))
            ref_np = ref_img.squeeze().detach().cpu()
            save_img(ref_np, f"ref/{idx}.png")
            ref_np = ref_np.numpy()
            ref_np = np.transpose(ref_np, (1, 2, 0))
            raw_np = raw_img

            curr_psnr = psnr(ref_np, enhanced_np)
       #     //curr_ssim = torchSSIM(ref_np, enhanced_np)
       #      curr_psnr = curr_psnr.cpu()
            psnr_values.append(curr_psnr)
      #      //ssim_values += curr_ssim

            # Print progress


    psnr_values = np.array(psnr_values)
    ssim_values = np.array(ssim_values)

    avg_psnr = psnr_values.sum()/len(psnr_values)
    #avg_ssim = ssim_values/len(test_loader)

    print(f"Evaluation Results:")
    print(f"Average PSNR: {avg_psnr:.2f} dB")


    with open(f'{results_dir}/metrics.txt', 'a') as f:
        f.write(f"Model: {model_path}\n")
        f.write(f"Average PSNR: {avg_psnr:.2f} dB\n")


Evaluation Results:
Average PSNR: 22.70 dB


In [43]:
from utils import save_img

import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
# from skimage.metrics import structural_similarity as ssim
import torch
from pytorch_msssim import ssim

import os

model_path = f"fflMix-0.0002-NewBigModel-1746968864.9918458-Wavelet/best_spectral_transformer_148.pth"
raw_dir = "../uw_data/uw_data/train/a"
ref_dir = "../uw_data/uw_data/train/b"
test_raw_dir = "../uw_data/uw_data/test/a"
test_ref_dir = "../uw_data/uw_data/test/b"
batch_size=1
num_workers=4


device = "cuda"
#model = SpectralTransformer().to(device)
#model = NewModel().to(device)
model = NewBigModel().to(device)
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
# model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

u45_dataset = get_U45_dataset("u")
u45_loader = torch.utils.data.DataLoader(u45_dataset, batch_size=batch_size)

for  idx,raw_img in enumerate(u45_loader):

     raw_img = raw_img[0].to(device)
     enhanced_img = model(raw_img)
     save_img(enhanced_img.squeeze().detach().cpu(), f"uTest/{idx}.png")

In [44]:
import os
import numpy as np
import cv2

from uqim_utils import getUIQM
def calculate_UIQM(image_path, resize_size=(256, 256)):
    image_list = os.listdir(image_path)
    uiqms = []

    for img in image_list:
        image = os.path.join(image_path, img)

        image = cv2.imread(image)
        image = cv2.resize(image, resize_size)

        # calculate UIQM
        uiqms.append(getUIQM(image))
    return np.array(uiqms)
UIQM_measures = calculate_UIQM('uTest', resize_size=(256, 256))
print("UIQM on {0} samples {1} ± {2}".format(len(UIQM_measures), np.round(np.mean(UIQM_measures), 3), np.round(np.std(UIQM_measures), 3)))

UIQM on 45 samples 4.33 ± 3.056


In [25]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity, peak_signal_noise_ratio
from SSIM_PIL import compare_ssim
from pytorch_msssim import ssim
from PIL import Image

def calculate_metrics_ssim_psnr(generated_image_path, ground_truth_image_path, resize_size=(256, 256)):
    generated_image_list = os.listdir(generated_image_path)
    error_list_ssim, error_list_psnr = [], []

    for img in generated_image_list:
        label_img = img
        generated_image = os.path.join(generated_image_path, img)
        ground_truth_image = os.path.join(ground_truth_image_path, label_img)
        gt = Image.open(ground_truth_image)
        gen_img = Image.open(generated_image)
        generated_image = cv2.imread(generated_image)
        generated_image = cv2.resize(generated_image, resize_size)

        ground_truth_image = cv2.imread(ground_truth_image)
        ground_truth_image = cv2.resize(ground_truth_image, resize_size)


        # calculate SSIM

        ssim_loss = compare_ssim(gen_img,gt)
        error_list_ssim.append(ssim_loss)

        generated_image = cv2.cvtColor(generated_image, cv2.COLOR_BGR2GRAY)
        ground_truth_image = cv2.cvtColor(ground_truth_image, cv2.COLOR_BGR2GRAY)

        # calculate PSNR
        error_psnr = peak_signal_noise_ratio(generated_image, ground_truth_image)
        error_list_psnr.append(error_psnr)

    return np.array(error_list_ssim), np.array(error_list_psnr)

SSIM_measures, PSNR_measures = calculate_metrics_ssim_psnr('test','ref')
PSNR_measures = np.sort(PSNR_measures)[4:]
print("SSIM on {0} samples {1} ± {2}".format(len(SSIM_measures), np.round(np.mean(SSIM_measures), 3), np.round(np.std(SSIM_measures), 3)))
print("PSNR on {0} samples {1} ± {2}".format(len(PSNR_measures), np.round(np.mean(PSNR_measures), 3), np.round(np.std(PSNR_measures), 3)))

No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
No module named 'pyopencl'
N

In [13]:
root_dirs_test = {
    'input': ['uw_data/uw_data/test/a'],
    'reference': ['uw_data/uw_data/test/b']
}
dataset_test = get_dataset(root_dirs_test)

data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=1, shuffle=True)

In [39]:
from torchvision import transforms
import torch
import os
from PIL import Image

class U45Dataset(torch.utils.data.Dataset):
    def __init__(self, input_dirs,transform=None):
        """
        Args:
            input_dirs (list): List of directories containing input images.

            transform (callable, optional): Optional transform to be applied to input images.

        """
        self.input_dirs = input_dirs

        self.transform = transform

        valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')

        # Gather image paths from multiple directories
        self.input_images = []
        self.reference_images = []

        input_files = sorted([f for f in os.listdir(input_dirs) if f.lower().endswith(valid_extensions)])


            # Ensure filenames match, skip if not
        for file in input_files:
                self.input_images.append(os.path.join(input_dirs, file))

    def __len__(self):
        return len(self.input_images)

    def __getitem__(self, idx):
        # Load images and apply transformations

        input_image = Image.open(self.input_images[idx]).convert("RGB")


        input_image = self.transform(input_image)
        return input_image,idx

def get_U45_dataset(input_dir):
    """
    Args:
        root_dirs (dict): Dictionary containing dataset folder paths with 'input' and 'reference' keys.
                          Example: {'input': ['/path/to/uw_data', '/path/to/LUSI'],
                                    'reference': ['/path/to/ref_data1', '/path/to/ref_data2']}
    """
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    input_dirs = input_dir
    return U45Dataset(input_dirs, transform )

In [12]:
from torchvision import transforms
import torch
import os
from PIL import Image

class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, input_dirs, reference_dirs, transform=None, transformH=None):
        """
        Args:
            input_dirs (list): List of directories containing input images.
            reference_dirs (list): List of directories containing reference images.
            transform (callable, optional): Optional transform to be applied to input images.
            reference_transform (callable, optional): Optional transform to be applied to reference images.
        """
        self.input_dirs = input_dirs
        self.reference_dirs = reference_dirs
        self.transform = transform
        self.transformH =transformH
        valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')

        # Gather image paths from multiple directories
        self.input_images = []
        self.reference_images = []
        for input_dir, reference_dir in zip(input_dirs, reference_dirs):
            input_files = sorted([f for f in os.listdir(input_dir) if f.lower().endswith(valid_extensions)])
            reference_files = sorted([f for f in os.listdir(reference_dir) if f.lower().endswith(valid_extensions)])

            # Ensure filenames match, skip if not
            for file in input_files:
                if file in reference_files:
                    self.input_images.append(os.path.join(input_dir, file))
                    self.reference_images.append(os.path.join(reference_dir, file))

    def __len__(self):
        return len(self.input_images)

    def __getitem__(self, idx):
        # Load images and apply transformations

        input_image = Image.open(self.input_images[idx]).convert("RGB")
        reference_image = Image.open(self.reference_images[idx]).convert("RGB")

        input_image = self.transform(input_image)

        reference_imageL = self.transform(reference_image)

        reference_imageH = self.transformH(reference_image)

        return input_image, reference_imageL,reference_imageH, idx

def get_dataset(root_dirs):
    """
    Args:
        root_dirs (dict): Dictionary containing dataset folder paths with 'input' and 'reference' keys.
                          Example: {'input': ['/path/to/uw_data', '/path/to/LUSI'],
                                    'reference': ['/path/to/ref_data1', '/path/to/ref_data2']}
    """
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),

    ])
    transformH = transforms.Compose([
        transforms.Resize((256*2, 256*2)),
        transforms.ToTensor(),

    ])
    input_dirs = root_dirs['input']
    reference_dirs = root_dirs['reference']
    return CustomImageDataset(input_dirs, reference_dirs, transform,transformH )